<a href="https://colab.research.google.com/github/carvalheiracarlos/deep_leaning_notebooks/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [21]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data()

In [22]:
'''
Probably need to merge this X_train and X_test, 
than partition again in some better proportion(train: 80% | test: 20%) 
'''
print(f'Train Sets Shapes.....X_train: {X_train.shape}')
print(f'Train Sets Shapes.....y_train: {y_train.shape}')
print(f'Tests Sets Shapes.....X_test: {X_test.shape}')
print(f'Tests Sets Shapes.....y_test: {y_test.shape}')

Train Sets Shapes.....X_train: (25000,)
Train Sets Shapes.....y_train: (25000,)
Tests Sets Shapes.....X_test: (25000,)
Tests Sets Shapes.....y_test: (25000,)


In [23]:
'''
partition_size: size based on Lenght on X_train
                to divide validation and train sets on proportion of 20% - 80%
'''

partition_size = int(len(X_train) * 0.2)
validation_reviews, validation_labels = X_train[:partition_size], y_train[:partition_size]
train_reviews, train_labels = X_train[partition_size:], y_train[partition_size:]

print(f'Validation Sets: {validation_reviews.shape}, {validation_labels.shape}')
print(f'Train Sets: {train_reviews.shape}, {train_labels.shape}')

Validation Sets: (5000,), (5000,)
Train Sets: (20000,), (20000,)


In [30]:
class DenseNet:
  @staticmethod
  def build(input_shape, 
            n_classes, 
            activation='relu', 
            weights_path=None):

    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(64, activation=activation, input_shape=input_shape))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

In [25]:
n_epochs = 200
ini_learning_rate = 1e-3
batch_size=128

In [33]:
train_model = DenseNet.build(input_shape=train_reviews.shape, 
                             n_classes=2, 
                             activation='relu', 
                             weights_path=None)

opt = tf.keras.optimizers.Adam(learning_rate=ini_learning_rate,
                            beta_1=0.9, 
                            beta_2=0.999, 
                            epsilon=1e-08,
                            decay=ini_learning_rate/n_epochs)

train_model.compile(loss='binary_crossentropy', 
                    optimizer=opt, 
                    metrics=[tf.keras.metrics.binary_accuracy])
train_model.summary()
tensorboard = tf.keras.callbacks.TensorBoard('./logs/fit/',
                                          write_images=True)

callbacks_list = [tensorboard]

fit_history = train_model.fit(x=train_reviews,
                              y=train_labels,
                              epochs=n_epochs,
                              validation_data=(validation_reviews, validation_labels),
                              validation_freq=1,
                              batch_size=batch_size,
                              callbacks=callbacks_list,
                              verbose=1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               10240512  
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 2)                 1026      
                                                                 
Total params: 10,241,538
Trainable params: 10,241,538
Non-trainable params: 0
_________________________________________________________________


ValueError: ignored